<a href="https://colab.research.google.com/github/bharatbajoria/Summer-Internship/blob/master/Automatic_Topic_labelling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
def automatic_labels(idealtopics,wikipedia,RegexpTokenizer,model1,stopwords,repetitions):
  tokenizer = RegexpTokenizer(r'\w+')
  en_stop = set(stopwords.words('english'))
  en_stop.add('the')#The is not in stopwords
  
  dup_model=model1
  
  topic_count=1
  for reruns in range(repetitions):
    print("Repetition:-",reruns+1)
    Automatic_labels=[]
    for x in idealtopics:

      print("Finding label for Topic:",topic_count)
      topic_count+=1
      
      contents=[]
      title_vector=[]   
      index_topic=[]
      #ndex_topic[0][1] # index_topic[i] is a list of titles obtained for word-1
      ## index_topic[i][j] is a list of words in j-th title obtained for word-1
      word_limit=20
      x=x[:word_limit]
      count=0
      k=0
      for j in x:
        contents=[]
        index_word=[]
        file_title=wikipedia.search(j,results=10,suggestion= True)
        print("\n\nTitle for word :", j)
        print(file_title[0])
        print(count)
        
        for l in range(len(file_title[0])):
          continue_var=0
          try:
            y = wikipedia.search(file_title[0][l])[2]
            y = wikipedia.page(y)
        # except wikipedia.DisambiguationError as e:
          #  s = e.options[0] 
          # y = wikipedia.page(s)
          except :             
            continue_var=1            

          if (continue_var==0 or type(y)=='str'):       
            token=tokenizer.tokenize(y.content)
            token=[i for i in token if(not(str(i).isdigit() or not(str(i).isalpha())) and len(str(i)) > 2 )]
            token=[i.lower() for i in token if( i not in en_stop)]
            contents.append(token)
          count+=1

        
          
        dup_model.build_vocab(contents,update= True)#content is a 2-d list
        dup_model.train(contents, total_examples=len(contents),epochs=2)
        for i in range(len(file_title[0])):
          
          
          y= file_title[0][i].lower()
          z=y.split()
          for m in range(len(z)):
            index=[]
            p=list(z[m])
            for k in range(len(p)):
              if (p[k]=="-" or p[k]==","):
                p[k]=" "

              if not(p[k].isalpha()):
                index.append(k)
            for k in range(len(index)):
              p.pop(index[len(index)-k-1])
            
            k=''
            for w in p:
              k+=w
            z[m]=k

          z=[k for k in z if not(k in en_stop or k== j or k=="" or k==",")]
          
          index_word.append(z)
          dup_model.build_vocab(index_word,update= True)
          dup_model.train(index_word, total_examples=len(contents),epochs=2)
        
          
        index_topic.append(index_word)

      max_score=0
      max_list=[]
      for i in range(len(index_topic)):
        o=0
        for j in range(len(index_topic[i])):
          o=0
          for k in range(len(index_topic[i][j])):
            for p in range(len(x)):
              o+=dup_model.similarity(index_topic[i][j][k],x[p])
          if len(index_topic[i][j])>0:
            o= o/len(index_topic[i][j])
          if o>max_score:
            max_score=o
            max_list=[i,j]

      file_title=wikipedia.search(x[max_list[0]],results=10,suggestion= False)
      Automatic_labels.append(file_title[max_list[1]])

  return Automatic_labels